<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
#!pip install imbalanced-learn -q
#!pip install swifter - q

#!pip install lightgbm
#!pip install xgboost

In [2]:
import pandas as pd
from tqdm import notebook
import re
from time import time 
#import nltk
import numpy as np
from numpy.random import default_rng
import transformers
from sklearn.model_selection import train_test_split,cross_val_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import f1_score

from __future__ import unicode_literals, print_function
from spacy.lang.en import English

In [3]:
from imblearn.under_sampling import RandomUnderSampler

In [4]:
from sklearn.model_selection import TunedThresholdClassifierCV
from sklearn.model_selection import FixedThresholdClassifier

In [5]:
import torch
import transformers

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
from xgboost import XGBClassifier

In [8]:
import swifter

In [9]:
from sklearn.metrics import classification_report

In [ ]:
from nltk.tokenize import sent_tokenize

In [ ]:
import nltk

In [383]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\LeeCo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [10]:
#import spacy
#from nltk.corpus import stopwords
#nltk.download('stopwords')
#from spacy.pipeline import Lemmatizer
#!python -m spacy download en_core_web_sm
#nlp = spacy.load("en_core_web_sm")

In [11]:
#tokenizer = transformers.BertTokenizer(
#    vocab_file='datasets/ds_bert/vocab.txt')

In [386]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')#'distilbert-base-uncased')

In [13]:
#nlp = English()
#nlp.add_pipe('sentencizer')

In [352]:
tokenizer

BertTokenizer(name_or_path='bert-large-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [15]:
tokenizer.encode('He ran. He fell.')

[101, 2002, 2743, 1012, 2002, 3062, 1012, 102]

In [16]:
tokenizer.added_tokens_encoder

{'[PAD]': 0, '[UNK]': 100, '[CLS]': 101, '[SEP]': 102, '[MASK]': 103}

In [17]:
for enc in tokenizer.encode('He ran. He fell. Aaaaaaaaaaa...'):
    display((enc,tokenizer.decode([f'{enc}'])))

(101, '[CLS]')

(2002, 'he')

(2743, 'ran')

(1012, '.')

(2002, 'he')

...

(11057, '##aa')

(1012, '.')

(1012, '.')

(1012, '.')

(102, '[SEP]')

In [18]:
RANDOM_STATE = 88

## Подготовка

In [19]:
start = time()
#df['text'].apply(lambda x: len(str(x).split()))
end = time()
end-start

5.5789947509765625e-05

In [20]:
url = 'https://code.s3.yandex.net/datasets/toxic_comments.csv'

In [21]:
df = pd.read_csv(url)

In [22]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [24]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [25]:
df['text'].duplicated().sum()

np.int64(0)

In [26]:
df.isna().sum()

text     0
toxic    0
dtype: int64

In [27]:
def clean_text(string):
    string = re.sub(r'[^a-z]', ' ', string, flags=re.IGNORECASE)
    string = string.split()
    string = ' '.join(string)
    return string.lower()

In [28]:
df['clean_text'] = df['text'].apply(lambda x:clean_text(x))

In [29]:
df['clean_text'].duplicated().sum()

np.int64(1323)

In [30]:
#df.drop_duplicates(subset='clean_text', inplace=True)

In [31]:
df['text'].duplicated().sum()

np.int64(0)

In [32]:
df['text'].str.lower().drop_duplicates(inplace=True)

In [33]:
df.reset_index(drop=True, inplace=True)

def sents(string):
    doc = nlp(string)
    sentences = [sent.text.strip() for sent in doc.sents]
    return sentences

In [35]:
df['word_count'] = df['text'].apply(lambda x: x.count(' ') + 1)

In [36]:
#df['sent_count'] = df['text'].apply(lambda x: len(sents(x)))

In [37]:
df['len'] = df['text'].apply(lambda x: len(x))

In [364]:
df[df['len']==df['len'].max()].head(10)

,text,toxic,clean_text,word_count,len
4167,ii CAN STILL POST WITH THIS COMPUTER...I SAID ...,1,ii can still post with this computer i said bl...,773,5000
6190,SECURITYFUCK HOMELAND SECURITYFUCK HOMELAND SE...,1,securityfuck homeland securityfuck homeland se...,455,5000
8685,BALLS BALLS BALLS BALLS BALLS BALLS BALLS BALL...,1,balls balls balls balls balls balls balls ball...,834,5000
9651,"FUCK YOU, YOU ATHEIST CUNT! FUCK YOU, YOU ATHE...",1,fuck you you atheist cunt fuck you you atheist...,894,5000
10746,TOMMY2010 IS A NIGGER TOMMY2010 IS A NIGGER TO...,1,tommy is a nigger tommy is a nigger tommy is a...,909,5000
12869,I OWN THIS PAGE YOU FOOLS!! YOU SHOULD FEAR ME...,1,i own this page you fools you should fear me i...,965,5000
16612,'''Hi Wikipedia!!Hi Wikipedia!!Hi Wikipedia!!H...,0,hi wikipedia hi wikipedia hi wikipedia hi wiki...,358,5000
18151,Hey guys I love chicken rice=)Hey guys I love ...,0,hey guys i love chicken rice hey guys i love c...,835,5000
22334,I fuck niggas! I fuck niggas! I fuck niggas! I...,1,i fuck niggas i fuck niggas i fuck niggas i fu...,1001,5000
25647,"""....would you prefer expressing my frustratio...",0,would you prefer expressing my frustration in ...,946,5000


In [38]:
df['text'][154485]

'"\n\n ""Disappointing Players for Charlton"" \n\nI added in a section on the April Fools prank in the Crest\'s section, as well as splitting the directors of the boardroom into two sections, Charlton Athletic Ltd & CAFC PLC.\n\n"'

In [39]:
#df['text'][112719]

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   text        159292 non-null  object
 1   toxic       159292 non-null  int64 
 2   clean_text  159292 non-null  object
 3   word_count  159292 non-null  int64 
 4   len         159292 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 6.1+ MB


In [41]:
#df['text_lower'] = df['mid_text'].str.lower()
#df.drop_duplicates(subset='text_lower', inplace=True)
#df.reset_index(drop=True, inplace=True)

In [42]:
def distribution_plt(df_list, name, val_list, plt_xlim=1):

    x_max = min([series.max() for series in df_list])* plt_xlim

    fig, ax = plt.subplots(2,1,figsize=(10,8))
    ax[0].boxplot([df for df in df_list], vert=False)

    ax[0].set_yticklabels(val_list)
    ax[0].set_xlabel(name)
    ax[0].set_xlim([0, x_max])

    ax[1].hist([df for df in df_list], histtype='stepfilled', bins=500)
    ax[1].set_ylabel('частота')
    ax[1].set_xlabel(name)
    ax[1].set_xlim([0, x_max])

    
    plt.suptitle(f'Ящики с усами и гистограмма для признака "{name}"')
    plt.legend(val_list);
    #plt.xlim((0,x_max))

In [43]:
name_dict = {'word_count' : 'количество слов',
 'sent_count' : 'количество предложений',
 'len' : 'количество символов'}

for col in ['word_count', 'sent_count', 'len']:
    
    distribution_plt(
        
        [df[df['toxic']==1][col], df[df['toxic']==0][col]],
        
        name_dict[col],
        
        ['токсичные\nкомментарии', 'не токсичные\nкомментарии'],
        
        0.1)

In [44]:
df[df['text'].str.contains('I ass. I ass.')]['text']

4704    do go fuck off bastard\nDo Yyou Have a life?\n...
Name: text, dtype: object

In [289]:
def logy(feats, target):
    lr = LogisticRegression(random_state=RANDOM_STATE,max_iter=1000,class_weight='balanced')
    lr.fit(feats, target)
    scores = cross_val_score(lr, feats, target, cv=5, scoring='f1_macro')
    return scores

In [46]:
df.shape

(159292, 5)

In [47]:
4096*16

65536

In [406]:
samp = df.sample(n=6000, random_state=RANDOM_STATE)

In [407]:
X_train, X_test, y_train, y_test = train_test_split(
    samp['text'],
    samp['toxic'],
    test_size = 0.5,
    random_state = RANDOM_STATE,
    stratify = samp['toxic'])

In [408]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3000,), (3000,), (3000,), (3000,))

In [390]:
def chunker(string):
    
    sents_ = sent_tokenize(string)

    tokenized_sents = [tokenizer.encode(sent, add_special_tokens=True) for sent in sents_]

    usents=[]
    
    for sent in tokenized_sents:
        if sent not in usents:
            usents.append(sent)
    
    if len(usents)>1:
       result =  sum(usents, [])
    elif len(usents)==1:
        result = usents[0]
    else:
        result = tokenized_sents[0]
        
    
    
    return [101] + result[1:-1][:510] + [102]

In [384]:
len(chunker(df['text'][16612]))

14

def chunker(string):
    
    sents_ = sents(string)
    
    tokenz = tokenizer(sents_, add_special_tokens=True, padding=True, return_tensors='np')


    model = transformers.BertModel.from_pretrained('bert-large-uncased')#'distilbert-base-uncased')
    
    with torch.no_grad():
                    batch_embeddings = model(torch.LongTensor(tokenz['input_ids']), attention_mask=torch.LongTensor(tokenz['attention_mask']))
    #transformers.from_pretrained(distilbert-base-uncased)
    
    #batch_embeddings[0][:,0,:]
    
    return np.mean(batch_embeddings[0][:,0,:].numpy(),axis=0)

In [281]:
chunker(df['text'][4700])

array([-1.06014431e-01, -6.21548295e-02, -1.01379892e-02, -1.68441549e-01,
        4.32150885e-02, -1.96783051e-01,  9.86524299e-02,  4.27524507e-01,
       -2.31145293e-01, -2.97342479e-01, -4.49013636e-02, -1.79771557e-01,
       -1.97978646e-01,  4.04549897e-01,  3.10555249e-02,  3.12551588e-01,
       -9.21233743e-02,  2.11014494e-01,  1.90578029e-01, -1.21308025e-02,
       -9.72667113e-02, -1.98384970e-01, -1.23006245e-02,  9.30938348e-02,
       -1.51105355e-02, -3.25137191e-03,  1.03774011e-01, -1.01887979e-01,
       -1.19080646e-02,  8.76610875e-02,  6.72649816e-02,  1.24185801e-01,
       -1.93346590e-01, -1.08881399e-01,  5.38776512e-04, -1.28543496e-01,
        1.45829124e-02, -6.01367913e-02, -1.17812902e-02,  8.68727118e-02,
       -1.45049170e-01,  1.06997609e-01,  7.22344443e-02,  3.60909812e-02,
       -2.58618314e-02, -1.04638956e-01, -2.39289665e+00, -1.21881008e-01,
       -1.28574058e-01, -2.63087422e-01,  8.48741904e-02, -2.38394793e-02,
        5.67158405e-03,  

In [385]:
class BertTokenizer(object):

    def __init__(
        self,
        #tokenizer,
        #text=[],
        text,
        batch_size=1):
        
        self.text = text
        self.batch_size = batch_size
        self.model_class ,self.pretrained_weights = (
            transformers.BertModel, 'bert-base-uncased')

        self.model = self.model_class.from_pretrained(self.pretrained_weights)

    
    def get(self):

        
        tokenized = self.text.swifter.apply(lambda x: chunker(x))
        
        max_len = 0
        
        for tok in tokenized.values:
            if len(tok) > max_len:
                max_len = len(tok)

        padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
        attention_mask = np.where(padded != 0, 1, 0)
        
        batch_size = self.batch_size
        
        embeddings = []
        
        for i in notebook.tqdm(range(len(padded) // batch_size)):
                batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
                attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
                
                with torch.no_grad():
                    batch_embeddings = self.model(batch, attention_mask=attention_mask_batch)
                
                embeddings.append(batch_embeddings[0][:,0,:].numpy())
    
        features = np.concatenate(embeddings)

        return features

In [330]:
X_train.shape[0]/30

1000.0

In [365]:
len(tokenizer.encode(df['text'][16612]))

1432

In [363]:
len(chunker(df['text'][6190]))

512

In [411]:
X_train.shape[0]

3000

In [409]:
dembs_train = BertTokenizer(text=X_train, batch_size=30).get()

Pandas Apply:   0%|          | 0/3000 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1067 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/100 [00:00<?, ?it/s]

In [392]:
trial = BertTokenizer(text=X_train, batch_size=50).get()

Pandas Apply:   0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

In [410]:
logy(dembs_train, y_train)

array([0.83608914, 0.80592992, 0.80231409, 0.79755344, 0.82055295])

In [226]:
logy(dembs_train, y_train)

array([0.80789152, 0.80608067, 0.80544205, 0.79700029, 0.81400535])

In [ ]:
#0.80789152, 0.80608067, 0.80544205, 0.79700029, 0.81400535

In [ ]:
dembs_test = BertTokenizer(text=X_test, batch_size=30).get()

In [94]:
from sklearn.utils.class_weight import compute_sample_weight
sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=y_train #provide your own target name
)

sample_weights

array([0.55718584, 0.55718584, 0.55718584, ..., 0.55718584, 4.87171159,
       0.55718584], shape=(30000,))

In [95]:
y_train.value_counts(normalize=1)

toxic
0    0.897367
1    0.102633
Name: proportion, dtype: float64

In [58]:
def f1_eval(y_true, y_pred,):
    #y_true = dtrain.get_label()
    #err = 1 - f1_score(grid_pred, y_test, average=None)[1]
    err = f1_score(y_pred, y_true, average=None)[1]
    return  err

In [133]:
def f1_xgb(y_pred, dtrain):
    y_true = dtrain.get_label()
    err =1 - f1_score(y_true, y_pred, average=None)[1]

    return 'err', err

In [134]:
from sklearn.metrics import make_scorer
my_func = make_scorer(f1_eval, greater_is_better=True)

In [135]:
y_train.value_counts(normalize=1)

toxic
0    0.897367
1    0.102633
Name: proportion, dtype: float64

In [205]:
debs_no_dups = pd.DataFrame(dembs_train).join(pd.DataFrame(y_train.reset_index()['toxic']))

In [206]:
debs_no_dups.drop_duplicates(inplace=True)

In [208]:
debs_no_dups

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,toxic
0,0.054554,0.032030,0.120778,-0.114607,-0.075923,-0.136677,0.020189,0.225854,-0.135898,-0.171853,...,-0.101006,0.080337,-0.064391,0.177604,0.002480,-0.193309,-0.046049,0.192293,0.222578,0
1,0.026916,0.070370,-0.041623,-0.070733,0.077861,-0.232926,0.148254,0.412093,-0.169133,-0.332854,...,-0.193595,0.064336,0.031726,0.189356,0.086667,-0.165502,-0.025272,0.250942,0.280510,0
2,-0.040808,-0.026892,0.116677,-0.078166,-0.035422,-0.195369,0.161824,0.360852,-0.162770,-0.091233,...,-0.048423,-0.054734,-0.057439,0.073722,-0.020744,-0.099020,-0.099139,0.271790,0.353031,0
3,-0.051494,-0.118144,0.135466,-0.082494,-0.017335,-0.195243,-0.063485,0.258741,-0.119786,-0.242215,...,-0.112844,-0.016042,-0.090609,0.245084,-0.206943,-0.137903,-0.286003,0.384027,0.331157,0
4,-0.015131,0.042579,-0.012480,-0.056407,-0.093443,-0.059389,0.056314,0.219939,-0.263359,-0.179751,...,-0.104669,-0.016733,-0.062705,0.171592,0.114356,-0.172997,-0.032023,0.278494,0.177390,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,0.044590,0.001134,-0.040821,-0.078365,-0.194404,-0.255145,0.224017,0.357888,-0.242456,-0.024003,...,-0.086306,0.034925,-0.199785,0.083316,-0.010829,-0.052159,0.016477,0.397497,0.419192,0
29996,-0.355209,-0.020035,-0.012108,-0.138657,-0.177016,-0.136832,0.035712,0.388245,-0.168578,-0.179795,...,-0.077247,-0.042935,0.049135,0.111670,-0.095760,-0.070035,-0.103592,0.297826,0.507109,1
29997,-0.107784,-0.094219,0.132195,-0.157644,0.070041,-0.296841,0.093094,0.513813,-0.140068,-0.159314,...,0.010522,0.058193,-0.069481,0.265057,0.080197,-0.401160,-0.257445,0.245340,0.447246,0
29998,0.154565,-0.029695,0.086085,-0.100714,0.036096,-0.258932,0.090568,0.402844,-0.084287,-0.194110,...,-0.118389,0.026744,-0.056654,0.186123,0.035641,-0.144441,0.006354,0.371069,0.255829,1


In [211]:
param_grid = [

    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE, class_weight='balanced')],
        'models__solver': ['newton-cg', 'liblinear'],
        #'models__class_weight':,
        #'models__C': [0.1, 1]#,
        #'models__penalty': ['l1', 'l2']
          
    },

#    {
#        'models': [SVC(random_state=RANDOM_STATE, class_weight='balanced')],
#        'models__degree': range(3,8),
#        'models__kernel': ['poly','rbf','sigmoid']
#    },
    {
        'models': [XGBClassifier(
            random_state=RANDOM_STATE,
            scale_pos_weight = 0.897367
            
            )],
        'models__n_estimators':[500],
        'models__max_depth':[7],
        'models__learning_rate':[0.7,0.1],
        'models__eval_metric': [f1_xgb]
    }
    #{
    #    'models': [LGBMClassifier(random_state=RANDOM_STATE, class_weight='balanced')],
    #    'models__n_estimators': [1000],
    #    'models__learning_rate': [0.1, 0.01, 0.03, 0.05, 0.07]
    #}
]

param_grid = [

    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE, class_weight='balanced')],
        'models__solver': ['newton-cg', 'liblinear'],
        #'models__class_weight':,
        'models__C': [0.1]#,
        #'models__penalty': ['l1', 'l2']
          
    }
]

In [212]:
pipe = Pipeline([('models',[LogisticRegression(random_state=RANDOM_STATE,class_weight='balanced')])])

In [213]:
grid_search = GridSearchCV(
    pipe, 
    param_grid, 
    cv=5,
    scoring=my_func,
    #scoring='f1_macro',
    n_jobs=-1
)
grid_search.fit(debs_no_dups.drop(['toxic'], axis=1), debs_no_dups['toxic'])

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('models',
                                        [LogisticRegression(class_weight='balanced',
                                                            random_state=88)])]),
             n_jobs=-1,
             param_grid=[{'models': [LogisticRegression(class_weight='balanced',
                                                        random_state=88)],
                          'models__solver': ['newton-cg', 'liblinear']},
                         {'models': [XGBClassifier(base_score=None,
                                                   booster=None, callbacks=None,
                                                   colsample_bylevel=None,
                                                   c...
                                                   max_leaves=None,
                                                   min_child_weight=None,
                                                   missing=nan,
                                                   monotone_constraints=None,
                                                   multi_strategy=None,
                                                   n_estimators=None,
                                                   n_jobs=None,
                                                   num_parallel_tree=None, ...)],
                          'models__eval_metric': [<function f1_xgb at 0x00000231F6CB9B20>],
                          'models__learning_rate': [0.7, 0.1],
                          'models__max_depth': [7],
                          'models__n_estimators': [500]}],
             scoring=make_scorer(f1_eval, response_method='predict'))

In [214]:
pd.DataFrame(grid_search.cv_results_)[['rank_test_score', 'mean_test_score','params']].sort_values('rank_test_score')#.iloc[1]['params']#.head(16)

,rank_test_score,mean_test_score,params
2,1,0.691408,"{'models': XGBClassifier(base_score=None, boos..."
3,2,0.690464,"{'models': XGBClassifier(base_score=None, boos..."
0,3,0.664492,{'models': LogisticRegression(class_weight='ba...
1,4,0.663929,{'models': LogisticRegression(class_weight='ba...


In [164]:
#grid_search.best_estimator_

In [215]:
grid_pred = grid_search.best_estimator_.predict(dembs_test)#[:X_train.shape[0]])

In [216]:
#probs = pd.DataFrame(grid_search.best_estimator_.predict_proba(dembs_test))

In [217]:
f1_score(grid_pred, y_test,average=None)

array([0.96984336, 0.69207773])

In [ ]:
y_test.shape

In [ ]:
dembs_test.shape

In [218]:
print(classification_report(grid_pred, y_test))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97     27727
           1       0.60      0.81      0.69      2273

    accuracy                           0.95     30000
   macro avg       0.79      0.89      0.83     30000
weighted avg       0.96      0.95      0.95     30000



In [219]:
classifier_tuned = TunedThresholdClassifierCV(
    grid_search.best_estimator_, scoring=my_func,
).fit(dembs_train, y_train)

In [220]:
ct_preds = classifier_tuned.predict(dembs_test)

In [221]:
f1_score(ct_preds, y_test)

0.7059925093632958

In [174]:
classifier_tuned.best_threshold_

np.float32(0.05050505)

In [ ]:
#f1_score(ct_preds, y_test, average=None)

In [185]:
classifier_other_threshold = FixedThresholdClassifier(
    grid_search.best_estimator_, threshold=0.15, response_method="predict_proba"
).fit(dembs_train, y_train)
f1_score(y_test, classifier_other_threshold.predict(dembs_test),average=None)#82

array([0.96847788, 0.71155473])

In [190]:
pd.DataFrame(dembs_train).duplicated().sum()

np.int64(16)

In [186]:
f1_score(y_test, classifier_other_threshold.predict(dembs_test))#82

0.7115547284723397

In [ ]:
cl_preds_adj = classifier_tuned.predict(dembs_val)

In [ ]:
f1_score(cl_preds_adj,y_val)

In [ ]:
res =pd.DataFrame({'a':grid_pred, 'b':test_sample['toxic']})

In [ ]:
pd.DataFrame(probs[0])

In [ ]:
y_test.shape

In [ ]:
pd.DataFrame({'proba':list(probs[1])},
              {'predicted':grid_pred})

In [ ]:
probs_res = pd.DataFrame(zip(y_test,probs[1],grid_pred),columns=['toxic', 'proba', 'pred'])

In [ ]:
probs_res[(probs_res['toxic']!=probs_res['pred']&(probs_res['toxic']==1))]['proba']

In [ ]:
X_test.iloc[1961]

In [ ]:
X_train.apply(lambda x: len(x)).hist()

In [ ]:
X_train.shape

In [ ]:
X_resample.shape

In [ ]:
X_train[X_train.apply(lambda x: len(x))>2000]

In [ ]:
res.shape

In [ ]:
df['text'].

In [ ]:
df[(df.index.isin(res[res['a']!=res['b']].index))&(df['toxic']==1)]['text'].apply(lambda x: tokenizer.decode(chunker(x)))#.sample()



#['b'].value_counts()

In [ ]:
tokenizer.decode(chunker(df.loc[128933]['text']))

In [ ]:
df.loc[128933]['text']

## Обучение

In [ ]:
#from phik.report import plot_correlation_matrix
#from phik import phik_matrix

In [ ]:
#df.info()

phik_overview = phik_matrix(df[['toxic','word_count']],verbose=False)

plot_correlation_matrix(
    phik_overview.values,
    x_labels=phik_overview.columns,
    y_labels=phik_overview.index,
    vmin=0, vmax=1, color_map='Blues',
    title=r'correlation $\phi_K$',
    fontsize_factor=1.5,
    figsize=(18, 14)
) 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df['text_lemm'], 
    df['toxic'], 
    random_state=RANDOM_STATE,
    stratify=df['toxic']
)

In [ ]:
sampler = RandomUnderSampler(random_state=RANDOM_STATE)
X_resample, y_resample = sampler.fit_resample(pd.DataFrame(X_train), pd.DataFrame(y_train)) 


In [ ]:
y_resample.value_counts()

In [ ]:
pd.DataFrame(y_test).value_counts()

## Выводы

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны